# A mixed FEM for Poisson equation

In [1]:
usewebgui = True
# If you want to use webgui, ensure you have it installed following
#    https://ngsolve.org/docu/latest/i-tutorials/index.html 
# Otherwise, set usewebgui to False above. 

import ngsolve as ng
from netgen.geom2d import SplineGeometry    
from ngsolve import dx, ds, div, x, y, exp, Integrate, sqrt, GridFunction

if usewebgui:
    import ngsolve.webgui
    from ngsolve.webgui import Draw
else:    
    import netgen.gui
    from ngsolve import Draw

First, let us create a domain with two subdomains. We shall use it to illustrate some of the properties of the H(div) space. 

In [2]:
geo = SplineGeometry()

#    3-------2------5
#    |       |      |
#    |       |      |
#    |       |      |
#    |       |      |
#    0-------1------4
#    


#       0        1        2         3        4      5
pts = [(0, 0), (1/2, 0), (1/2, 1), (0, 1), (1, 0), (1, 1)]
pn = [geo.AppendPoint(*p) for p in pts]

#     from, to, name,  left, right
lns = [(0,  1,  'bot', 1,    0), 
       (1,  2,  'mid', 1,    2), 
       (2,  3,  'top', 1,    0),
       (3,  0,  'lft', 1,    0),
       (1,  4,  'bot', 2,    0),
       (4,  5,  'rgt', 2,    0),
       (5,  2,  'top', 2,    0)
      ]

for p1, p2, bc, left, right in lns:
    geo.Append(["line", pn[p1], pn[p2]], 
               bc=bc, leftdomain=left, rightdomain=right)

geo.SetMaterial(1, "lftdom")
geo.SetMaterial(2, "rgtdom")
a=(1,3)
print(*a)

1 3


In [3]:
mesh = ng.Mesh(geo.GenerateMesh(maxh=0.3))
Draw(mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'draw_vol': N…

## Raviart-Thomas space

The lowest order Raviart-Thomas space can be generated in NGSolve by this:

In [4]:
R = ng.HDiv(mesh, order=0, RT=True)

Note its dimension:

In [5]:
R.ndof

43

In particular, note how this number equals the number of mesh edges. 

In [6]:
mesh.nedge

43

## Interpolation into the Raviart-Thomas space

Let's make up a vector field which is discontinuous across the middle interface between `lftdom` and `rgtdom`. 

In [9]:
v = mesh.MaterialCF({'lftdom': (x, y), 'rgtdom': (1-x, 1-y)})

Note that although the tangential component of `v` jumps across the interface, its normal component remains continuous.

In [10]:
Draw(v, mesh, 'v')

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

Note:
- In NGSolve's webgui, a vector field can be displayed using arrows if you follow the `Vectors -> show` check box.  Selecting from the `eval` control box, you can overlay the vector plot with individual components or the norm of the vector field. 

- In Netgen's GUI, the procedure is different: to display a vector field, check `Draw Surface Vectors` box in the `Visual` menu. To overlay a scalar contour plot, select the components using the `Scalar function` option.

The interpolation is performed using the `Set` method, which you have already seen as a method of an `H1` object. The interpolation process is different for different spaces, but they are all accessed by the `Set` method in NGSolve. The `Set` below uses H(div) interpolation.

In [11]:
vh = ng.GridFunction(R)
vh.Set(v)     # Compute interpolant of v
Draw(vh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

When the discontinuous field `v` with continuous normal component is interpolated to `vh`, we find that the interpolation process is (machine) exact, i.e., it only generates  (machine) zero error.

In [12]:
sqrt(Integrate((vh-v)**2, mesh))   # Error in RT interpolation

1.3319759440264468e-16

If you interpolate using two copies of `H1` to make up vector fields (available as `VectorH1` in NGSolve), you do not get (machine) zero error.

In [13]:
V = ng.VectorH1(mesh, order=1)
vvh = ng.GridFunction(V)
vvh.Set(v)
sqrt(Integrate((vvh-v)**2, mesh))

0.11687981282741002

Indeed, it is impossible for the `H1` interpolation to be exact since its result `vvh` must be a continuous vector field, which cannot equal the discontinuous field `v`.

## Visualize Raviart-Thomas shape functions

In [14]:
shape = ng.GridFunction(R, name='shape')
shape.vec[:] = 0
shape.vec[19] = 1
Draw(shape)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

You can check that the normal component of any shape function that you elect to plot  is continuous. Their tangential components need not be continuous. 

## Products of finite element spaces

Here is how you define product spaces.

In [15]:
R = ng.HDiv(mesh, order=0, RT=True)
W = ng.L2(mesh, order=0)

RW = ng.FESpace([R, W])

Note that the space `W` is just the space of piecewise constants on the mesh. Therefore its dimension equals  the number of mesh elements.

In [16]:
W.ndof

24

In [17]:
mesh.GetNE(ng.VOL)

24

## Assembly in two ways

Let's assemble the matrix of the mixed method for Poisson equation. We can do it in two ways. 


The first way is to make a "big" bilinear form in the product space. Note that trial and test functions in `RW` have two components.

In [18]:
q, u = RW.TrialFunction() 
r, v = RW.TestFunction()

B = ng.BilinearForm(RW)
B += (q*r - u*div(r) - div(q)*v) * dx
B.Assemble();

The second way is to make individual blocks and put them together. The blocks use different trial and test space, so notice the differences in  the syntax.

In [19]:
q, r = R.TnT()
u, v = W.TnT()

b = ng.BilinearForm(trialspace=R, testspace=W)
b += -div(q)*v * dx

a = ng.BilinearForm(R)
a += q*r * dx

b.Assemble(); a.Assemble();

Here is how you place these matrices into the proper place in a block partitioning:

In [20]:
BB = ng.BlockMatrix([[a.mat, b.mat.T], [b.mat, None]])

## Cross checking matrix vector products

As an exercise in becoming familiar with block matrices and component grid functions in NGSolve, let's compute the product of the assembled matrix `B.mat` and the block matrix `BB`  with the same vector and verify that we get the same result. We begin by setting some grid functions to get a vector on which the matrices can be applied. 

In [21]:
qh = GridFunction(R)
uh = GridFunction(W)
qh.Set((x, y))
uh.Set(x*y)

First we compute product using block matrix vector product facility. The above two grid functions can be made into a block vector as follows.

In [22]:
quvec = ng.BlockVector([qh.vec, uh.vec])

In [23]:
# Allocate space to store the matrix-vector product
BBqh = qh.vec.CreateVector()
BBuh = uh.vec.CreateVector()
BBquh = ng.BlockVector([BBqh, BBuh])

# Compute the product as a BlockVector
BBquh.data = BB * quvec

Next, we compute the product using the single assembled matrix `B.mat`. This matrix must act on grid functions of the product space `RW`, so we make one called `quh` below into which we  copy over the contents of the prior `qh` and `uh`.

In [24]:
# Allocate space for the result in a product FESpace
quh = GridFunction(RW)
quh.components[0].vec.data = qh.vec
quh.components[1].vec.data = uh.vec

# Matrix-vector multiply in a product FESpace GridFunction
Bquh = GridFunction(RW)
Bquh.vec.data = B.mat * quh.vec

Now we have the values of the matrix-vector products in a block vector `BBqu` and a vector of a grid function of a product finite element space. To compare their values, we can certainly print out the whole vector. We can also use the norm facility to quickly check that the  difference between the two vectors is zero.

In [25]:
d = BBqh.CreateVector()
d.data = BBqh - Bquh.components[0].vec
d.Norm()

5.971115260002713e-15

In [26]:
d.Norm()

5.971115260002713e-15

In [27]:
d = BBuh.CreateVector()
d.data = BBuh - Bquh.components[1].vec
d.Norm()

1.6675881526026532e-15

It's not a good idea to directly perform vector operations between a `BlockVector` and a regular NGSolve vector (in most cases NGSolve will raise an exception and will not allow you to do so). So if you work with block vectors, make sure you can access its component vectors to perform operations like the above.  

## Transferring to Scipy

Sparse matrices can be taken from NGSolve and given to Scipy. The blocks `a.mat` and `b.mat` that we made above can be made into scipy sparse matrix objects using the COO (coordinate) format.

In [28]:
from scipy.sparse import coo_matrix
import numpy as np
from scipy.linalg import svd

i, j, v = a.mat.COO()
A = coo_matrix((v, (i,j)))

i, j, v = b.mat.COO()
B = coo_matrix((v, (i,j)))

It is easy to perform matrix operations within numpy or scipy, e.g., computing the Schur complement.

In [29]:
Ainv = np.linalg.inv(A.toarray())
C = B @ Ainv @ B.T

Recall that invertibility of the Schur complement guarantees that the entire system is invertible (since $A$ is invertible in this mixed method). We compute the singular values and verify that all of them are positive. (For a small problem like this, computing the SVD is not expensive.)

In [30]:
u, s, vt = svd(C)
s

array([19.17215044, 18.20769681, 16.63066275, 16.19035552, 16.11365663,
       14.05393028, 13.78572502, 13.73565594, 12.553198  , 11.30239384,
       11.28320014, 10.91718695, 10.38573022,  7.32527422,  5.87721911,
        5.55235478,  5.06084957,  4.66526777,  3.75122339,  3.55823102,
        3.42653547,  2.19991415,  2.05785557,  0.85442697])

## Unstable pair

Recall that if the null space of $B^t$ is trivial, then 
$$
\begin{bmatrix}
A & B^t \\
B & 0 
\end{bmatrix}
$$
is invertible. If you use the Lagrange finite element space for fluxes and the primal variable, then you end up with a $B^t$ which has a nontrivial null space. 

In [31]:
from scipy.linalg import null_space

In [32]:
R = ng.VectorH1(mesh, order=1)
W = ng.H1(mesh, order=1)

q, r = R.TnT()
u, v = W.TnT()
b = ng.BilinearForm(trialspace=R, testspace=W)
b += -div(q)*v * dx
b.Assemble(); 

i, j, v = b.mat.COO()
B = coo_matrix((v, (i,j)))

We compute the null space using scipy.

In [33]:
Z = null_space(B.toarray().T)

The numpy array `Z` can be copied to a vector of coefficients of an NGSolve grid function. Then we can use NGSolve to visualize the kernel  function.

In [34]:
z = GridFunction(W, 'nullZ')
z.vec.data = Z[:, 1]
Draw(z)

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (arg0: ngsolve.la.BaseVector, arg1: ngla::DynamicVectorExpression) -> None

Invoked with: basevector, array([-0.07363282, -0.07363282, -0.07363282, -0.07363282, -0.07363282,
       -0.07363282,  0.31068214, -0.23704932,  0.31068214,  0.31068214,
        0.31068214, -0.23704932,  0.31068214, -0.23704932,  0.31068214,
        0.31068214, -0.07363282, -0.23704932, -0.07363282, -0.23704932])

## Solve a problem

Let us put $f=1$ on the left subdomain, $f=0$ elsewhere, and solve the Poisson equation in  mixed for on a finer mesh.

In [ ]:
mesh = ng.Mesh(geo.GenerateMesh(maxh=0.05))
f = mesh.MaterialCF({'lftdom': 1, 'rgtdom': 0})

In [ ]:
R = ng.HDiv(mesh, order=0, RT=True)
W = ng.L2(mesh, order=0)

In [ ]:
RW = ng.FESpace([R, W])
(q, u), (r, v) = RW.TnT()
B = ng.BilinearForm(RW)
B += (q*r - u*div(r) - div(q)*v) * dx
F = ng.LinearForm(f*v*dx)
B.Assemble(); F.Assemble();

In [ ]:
quh = GridFunction(RW, name='qu')
quh.vec.data = B.mat.Inverse() * F.vec
Draw(quh.components[1])
# Draw(quh.components[0])

Note how zero the solution approaches zero on the boundary even though none of the spaces used above imposed any essential boundary condition.


The same solution can be obtained using the block matrix facilities.

In [ ]:
q, r = R.TnT()
u, v = W.TnT()

b = ng.BilinearForm(trialspace=R, testspace=W)
b += -div(q)*v * dx
a = ng.BilinearForm(R)
a += q*r * dx
fW = ng.LinearForm(f*v*dx)
b.Assemble(); a.Assemble(); fW.Assemble();

In [ ]:
BB = ng.BlockMatrix([[a.mat, b.mat.T], [b.mat, None]])

In [ ]:
qh = GridFunction(R, name='q')
uh = GridFunction(W, name='u')

x = ng.BlockVector([qh.vec, uh.vec])
zero = qh.vec.CreateVector()
zero[:] = 0
FF = ng.BlockVector([zero, fW.vec])

In [ ]:
ng.solvers.MinRes(mat=BB, rhs=FF, sol=x, maxsteps=500, printrates=False)
Draw(uh)